In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [5]:
df=pd.read_csv("Fraud.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [7]:
le=LabelEncoder()
df['type']=le.fit_transform(df['type'])
df['nameOrig']=le.fit_transform(df['nameOrig'])
df['nameDest']=le.fit_transform(df['nameDest'])
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,757869,170136.0,160296.36,1662094,0.0,0.0,0,0
1,1,3,1864.28,2188998,21249.0,19384.72,1733924,0.0,0.0,0,0
2,1,4,181.00,1002156,181.0,0.00,439685,0.0,0.0,1,0
3,1,1,181.00,5828262,181.0,0.00,391696,21182.0,0.0,1,0
4,1,3,11668.14,3445981,41554.0,29885.86,828919,0.0,0.0,0,0


In [8]:
X=df.drop('isFraud',axis=1)
y=df['isFraud']
scaler = preprocessing.MinMaxScaler()
x_scaled = scaler.fit_transform(X)
df_scaled=pd.DataFrame(x_scaled,columns=X.columns)
df_scaled.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFlaggedFraud
0,0.0,0.75,0.000106,0.119287,0.002855,0.003233,0.610534,0.000000,0.0,0.0
1,0.0,0.75,0.000020,0.344545,0.000357,0.000391,0.636919,0.000000,0.0,0.0
2,0.0,1.00,0.000002,0.157738,0.000003,0.000000,0.161509,0.000000,0.0,0.0
3,0.0,0.25,0.000002,0.917359,0.000003,0.000000,0.143881,0.000059,0.0,0.0
4,0.0,0.75,0.000126,0.542392,0.000697,0.000603,0.304485,0.000000,0.0,0.0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_scaled, y, test_size=0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.35) 
lr_clf_none=LogisticRegression(penalty="none")
lr_clf_none.fit(X_train,y_train)
pred_none=lr_clf_none.predict(X_test)
accuracy=accuracy_score(y_test,pred_none)
print("Accuracy is ",accuracy*100)
print(classification_report(y_test,pred_none))

Accuracy is  99.922725753437
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906299
           1       0.89      0.46      0.61      2487

    accuracy                           1.00   1908786
   macro avg       0.95      0.73      0.80   1908786
weighted avg       1.00      1.00      1.00   1908786



In [10]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
kfld = []
  
for train_index, test_index in kf.split(X,y):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    lr_clf_none.fit(x_train_fold, y_train_fold)
    kfld.append(lr_clf_none.score(x_test_fold, y_test_fold))
kfld    

[0.999143434622844,
 0.9992133743646485,
 0.9992809565870663,
 0.9992165177238308,
 0.9991622947779374]

In [12]:
print(np.mean(cross_val_score(lr_clf_none, x_scaled, y, cv=kf)))

0.9992033156152654


In [14]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
Skfld = []
  
for train_index, test_index in skf.split(X,y):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    lr_clf_none.fit(x_train_fold, y_train_fold)
    Skfld.append(lr_clf_none.score(x_test_fold, y_test_fold))
Skfld    

[0.9991709390156885,
 0.9992212327626041,
 0.9991897991707819,
 0.999185869971804,
 0.9992063018064885]

In [15]:
print(np.mean(cross_val_score(lr_clf_none, x_scaled, y, cv=skf)))

0.9991948285454735


In [17]:
#regulaization
regl1=LogisticRegression(penalty="l1",solver="liblinear")
regl1.fit(X_train,y_train)
predl1=regl1.predict(X_test)
accuracy=accuracy_score(y_test,predl1)
print("Accuracy is ",accuracy*100)
print(classification_report(y_test,predl1))
#kfold
print("KFold is ",np.mean(cross_val_score(regl1, x_scaled, y, cv=kf)))
#StratifiedKFold
print("StratifiedKFold is ",np.mean(cross_val_score(regl1, x_scaled, y, cv=skf)))


Accuracy is  99.91675337099078
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906299
           1       0.91      0.40      0.56      2487

    accuracy                           1.00   1908786
   macro avg       0.96      0.70      0.78   1908786
weighted avg       1.00      1.00      1.00   1908786

KFold is  0.9991839839562948
StratifiedKFold is  0.9991838267883357


In [19]:
from sklearn.linear_model import Ridge 
ridge_reg=Ridge(alpha=50, max_iter=100,tol=0.1)
ridge_reg.fit(X_train,y_train)
ridge_reg.score(X_test,y_test)

0.04768426334967357

In [26]:
predict_val=[] 
predict_train=[]
min =[]
C_param_range = [0.1,1,5,10,30,50,80,100]
for i in C_param_range:
    lr_clf=LogisticRegression(C=i)
    lr_clf.fit(X_train,y_train)
    pred_val=lr_clf.predict(X_val)
    accuracy_val=accuracy_score(y_val,pred_val)
    err_val=1-accuracy_val
    predict_val.append(1-accuracy_val)
    pred_train=lr_clf.predict(X_train)
    accuracy_train=accuracy_score(y_train,pred_train)
    predict_train.append(1-accuracy_train)
best_lambda=C_param_range[np.argmin(predict_val)]
lr_clf=LogisticRegression(C=best_lambda)
lr_clf.fit(X_train,y_train)
pred_ridge=lr_clf.predict(X_test)
accuracy_ridge=accuracy_score(y_test,pred_ridge)
print("Accuracy of ridge is ",accuracy_ridge*100)
print(classification_report(y_test,pred_ridge))
print("KFold is",np.mean(cross_val_score(lr_clf, x_scaled, y, cv=kf)))
print("StratifiedKFold is",np.mean(cross_val_score(lr_clf, x_scaled, y, cv=skf)))

Accuracy of ridge is  99.91230027881596
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906299
           1       0.96      0.34      0.50      2487

    accuracy                           1.00   1908786
   macro avg       0.98      0.67      0.75   1908786
weighted avg       1.00      1.00      1.00   1908786

KFold is 0.9991536505401862
StratifiedKFold is 0.9991514501887588


In [28]:
lr_clf_l1=LogisticRegression(penalty="l1",solver="liblinear",C=500)
lr_clf_l1.fit(X_train,y_train)
pred_test=lr_clf_l1.predict(X_test)
accuracy_val=accuracy_score(y_test,pred_test)
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906299
           1       0.91      0.50      0.65      2487

    accuracy                           1.00   1908786
   macro avg       0.95      0.75      0.82   1908786
weighted avg       1.00      1.00      1.00   1908786

